# Download IMERG-Final data

In [1]:
import subprocess
import dask
from dask.distributed import Client, Semaphore
import tempfile
from pathlib import Path
import configparser
import pandas as pd
import os
import xarray as xr
import rioxarray as rxr
import requests

In [2]:
client = Client(n_workers=os.cpu_count())
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 64
Total threads: 64,Total memory: 187.56 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45927,Workers: 64
Dashboard: http://127.0.0.1:8787/status,Total threads: 64
Started: Just now,Total memory: 187.56 GiB
Comm: tcp://127.0.0.1:45593,Total threads: 1
Dashboard: http://127.0.0.1:37891/status,Memory: 2.93 GiB
Nanny: tcp://127.0.0.1:44287,


In [3]:
secrets = configparser.ConfigParser()
secrets.read('../../secrets/secrets.ini')

['../../secrets/secrets.ini']

In [ ]:
start_date = '2007-01-01'
end_date = '2021-09-30'
imerg_final_dir = Path("../../data-imerg_final-2010_2022/imerg_final")
download_dir = imerg_final_dir / "ss_method_zip"
output_dir = imerg_final_dir / "ss_method"

download_dir.mkdir(exist_ok=True)
output_dir.mkdir(exist_ok=True)

sem = Semaphore(max_leases=8)

def download_imerg_final(date, sem):
    date = pd.to_datetime(date)
    jan_1 = pd.to_datetime(f"{date.year}-01-01")
    imerg_final_id = ((date-jan_1) * 30).days

    if imerg_final_id <1000:
        if imerg_final_id <30:
            imerg_final_id = f'000{imerg_final_id}'
        elif imerg_final_id <100:
            imerg_final_id = f'00{imerg_final_id}'
        else:
            imerg_final_id = f'0{imerg_final_id}'

    link = f"https://arthurhouhttps.pps.eosdis.nasa.gov/gpmdata/{date.strftime('%Y')}/{date.strftime('%m')}/{date.strftime('%d')}/gis/3B-DAY-GIS.MS.MRG.3IMERG.{date.strftime('%Y%m%d')}-S000000-E235959.{imerg_final_id}.V06B.zip"
    fp_zip = (download_dir / date.strftime('%Y-%m-%d')).with_suffix('.zip')
    fp_dst = (output_dir / date.strftime('%Y-%m-%d')).with_suffix('.tif')

    cmd = [
        "wget",
        "-O",
        fp_zip,
        "--user",
        f"{secrets['imerg']['username']}",
        "--password",
        f"{secrets['imerg']['pwd']}",
        link,
        "--no-proxy"
    ]

    result = None
    # if not fp_zip.exists():
    with sem:
        response = requests.head(link,auth=(secrets["imerg"]["username"], secrets["imerg"]["pwd"]))
        if response.status_code == 200:
            result = subprocess.run(cmd, capture_output=True)
        else:
            print(f"[Download error] Data not available: {date}; {response.status_code}; ")

    with tempfile.TemporaryDirectory(dir=download_dir) as fp_temp:
        # unzip
        cmd = [
            "unzip",
            f'{fp_zip}',
            "-d",
            f'{fp_temp}'
        ]
        result2 = subprocess.run(cmd, capture_output=True)

        # convert to cog
        fp_temp_file = Path(fp_temp) / f"3B-DAY-GIS.MS.MRG.3IMERG.{date.strftime('%Y%m%d')}-S000000-E235959.{imerg_final_id}.V06B.total.accum.tif"
        cmd = [
            "gdal_translate",
            "-of",
            "COG",
            f"{fp_temp_file}",
            f"{fp_temp_file}_cog.tif"
        ]
        result3 = subprocess.run(cmd, capture_output=True)
        # mv
        cmd = [
            "mv",
            f'{fp_temp_file}_cog.tif',
            f'{fp_dst}'
        ]
        result4 = subprocess.run(cmd, capture_output=True)

        if result2.returncode != 0 or result3.returncode != 0 or result4.returncode != 0:
            print(f"Error: {date}")
            
                    
        return result, result2.returncode, result3.returncode, result4.returncode
    # return None, None, None, None

futures = []
# Download data from start_date to end_date
for date in pd.date_range(start_date, end_date):
    future = dask.delayed(download_imerg_final)(date, sem)
    futures.append(future)

# # Retry failed downloads
# failed_list = open('/water2/pdas47/2023_01_24-river-regulation/data-imerg_final-2010_2022/imerg_final/download-failed.txt').read().splitlines()
# for date in failed_list:
#     date = pd.to_datetime(date)
#     future = dask.delayed(download_imerg_final)(date, sem)
#     futures.append(future)

results = dask.compute(*futures)
results

In [22]:
@dask.delayed
def convert_to_da(fn, dst_dir, interp_lat_lon=None):
    da = rxr.open_rasterio(fn, masked=True, chunks=dict(x='auto', y='auto')).squeeze().drop('band').astype('float64')
    da.name = 'precip'
    time = pd.to_datetime(fn.stem)

    da = da.expand_dims('time')
    da = da.rename({'x': 'lon', 'y': 'lat'})
    da = da.assign_coords(
        time = ('time', [time]),
        lon = ('lon', da.lon.values.round(5)),
        lat = ('lat', da.lat.values.round(5))
    )

    if interp_lat_lon is not None:
        da = da.interp(lat=interp_lat_lon['lat'], lon=interp_lat_lon['lon'], method='nearest')

    da.attrs['units'] = 'mm'
    da.attrs['long_name'] = 'Precipitation'
    da.attrs['standard_name'] = 'precipitation'
    da.attrs['description'] = 'Accumulated precipitation estimated by IMERG Final Run'
    
    dst_fp = dst_dir / fn.name.replace('.tif', '.nc')

    da.to_netcdf(dst_fp)

In [23]:
existing_ds_fn = Path("../../data-cumberland-2010_2022/tennessee/basins/cumberland/pre_processing/nc/combined_data.nc")
existing_ds = xr.open_dataset(existing_ds_fn)
existing_ds

<xarray.Dataset>
Dimensions:  (time: 5091, lat: 35, lon: 89)
Coordinates:
  * time     (time) datetime64[ns] 2007-10-24 2007-10-25 ... 2021-09-30
  * lat      (lat) float64 37.47 37.41 37.34 37.28 ... 35.53 35.47 35.41 35.34
  * lon      (lon) float64 -88.28 -88.22 -88.16 -88.09 ... -82.91 -82.84 -82.78
Data variables:
    precip   (time, lat, lon) float64 ...
    tmax     (time, lat, lon) float64 ...
    tmin     (time, lat, lon) float64 ...
    wind     (time, lat, lon) float64 ...
    extent   (lat, lon) float64 ...

In [26]:
tif_dir = Path(f"../../data-imerg_final-2010_2022/imerg_final/ss_method")
dst_dir = Path(f"../../data-imerg_final-2010_2022/imerg_final/ss_method_nc")

tasks = [convert_to_da(fp, dst_dir, interp_lat_lon=dict(lat=existing_ds.lat, lon=existing_ds.lon)) for fp in tif_dir.glob("*.tif")]
tasks

[Delayed('convert_to_da-4e2bc15e-9e8f-4b62-adbe-6bc5717b6ad0'),
 Delayed('convert_to_da-be8fb098-a490-4384-8167-09d98205caa8'),
 Delayed('convert_to_da-a6bdcfdc-2649-4e87-93f3-88af863de6d1'),
 Delayed('convert_to_da-7b79423f-ca7c-4a5b-bc20-30e98d0ca5bc'),
 Delayed('convert_to_da-7b039645-817a-4528-94f7-c9f3cff21c7d'),
 Delayed('convert_to_da-e717723e-3418-4b87-a449-7344b60e59ea'),
 Delayed('convert_to_da-13e1ad83-0a63-45c2-a500-25e5346c3abb'),
 Delayed('convert_to_da-c136c9af-ca4d-43cd-84d7-d91816b05763'),
 Delayed('convert_to_da-09ceac47-0c33-48be-9555-512f9031699a'),
 Delayed('convert_to_da-f997a292-046b-4ec2-a845-45afad6a0587'),
 Delayed('convert_to_da-439e9558-4dc4-4bc4-ae5b-08d6f12963f1'),
 Delayed('convert_to_da-82f3c780-2cd8-4fff-bead-570cc20ee758'),
 Delayed('convert_to_da-a3d31eef-048a-4c4a-80c1-8dce72efacdc'),
 Delayed('convert_to_da-9150463c-956e-40c9-a833-f30ee269cdf2'),
 Delayed('convert_to_da-dc91e8eb-bd46-430d-a5a4-fbd5b9bbb76a'),
 Delayed('convert_to_da-601b3679-4bbd-43

In [28]:
dask.compute(*tasks)

(None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,